In [12]:
from typing import Tuple
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset
import torch
import pandas as pd

max_time_delta = 0
mean_iri = 0
iri_range = 0

class IRIDataset(Dataset):
    def __init__(self, data: pd.DataFrame, seq_length=10, padding_value=-1):
        global max_time_delta

        self.__raw_data = data
        self.__padding_value = padding_value

        self._inputs = []
        self._outputs = []

        # This will split each sequence (SHRP_ID, STATE_CODE) into sequences of length seq_length
        # that contain between 1 and seq_length-1 rows from the original dataset
        main_bar = self.__raw_data.groupby(["SHRP_ID", "STATE_CODE"])
#         main_bar = tqdm(self.__raw_data.groupby(["SHRP_ID", "STATE_CODE"]), leave=False, desc="Generating sequences")
        for group in main_bar:
            data = group[1].sort_values(["VISIT_DATE"], ascending=True)
            data["RELATIVE_TIME"] = pd.to_datetime(data["VISIT_DATE"])

            # subtract the first date from all dates to get a relative date
            data["RELATIVE_TIME"] = data["RELATIVE_TIME"] - data["RELATIVE_TIME"].iloc[0]
            data["RELATIVE_TIME"] = data["RELATIVE_TIME"].dt.days
            local_max = data["RELATIVE_TIME"].max()
            if local_max > max_time_delta:
                max_time_delta = local_max

            for i in range(2, len(data)):
                print('i:',i)
                tmp = pd.DataFrame(data.iloc[:i])
                print('tmp:')
                print(tmp)
                # Pad the sequence to length seq_length
                pad = pd.DataFrame(self.__padding_value, index=range(seq_length - len(tmp)), columns=tmp.columns)
                print('pad:')
                print(pad)
                tmp = pd.concat([pad, tmp])[-seq_length:]
                print('final:')
                print(tmp)

                inputs = tmp[['IRI_LEFT_WHEEL_PATH', 'IRI_RIGHT_WHEEL_PATH', 'RELATIVE_TIME']].to_numpy(dtype=float)
                inputs[-1, 0] = -1
                inputs[-1, 1] = -1
                
                print('inputs:')
                print(inputs)

                self._inputs.append(inputs)
                self._outputs.append(data.iloc[i-1][['IRI_LEFT_WHEEL_PATH', 'IRI_RIGHT_WHEEL_PATH']].to_numpy(dtype=float))
            
            data.drop(columns=["VISIT_DATE"], inplace=True)

    def __len__(self):
        return len(self._inputs)
    
    def __getitem__(self, idx):
        normalized = self._inputs.copy()[idx]
        normalized[:, 2] = normalized[:, 2] # / max_time_delta
        return torch.from_numpy(normalized.T).float(), torch.from_numpy(self._outputs[idx]).float()

In [13]:
import pandas as pd
raw_data = pd.read_parquet(path="../training_data/final_data.parquet")
# IRIDataset1 = IRIDataset()
IRIDataset(raw_data)

i: 2
tmp:
                   VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
SHRP_ID STATE_CODE                                                         
0001    48         1990-03-06               0.6904                0.6570   
        48         1992-03-17               0.6894                0.7312   

                    MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
SHRP_ID STATE_CODE                                                        
0001    48                89.222222        44.814815          20.363636   
        48                89.222222        44.814815          20.363636   

                    FREEZE_THAW_YR  TOTAL_ANN_PRECIP  TOTAL_SNOWFALL_YR  \
SHRP_ID STATE_CODE                                                        
0001    48                15.30303        859.939394          12.424242   
        48                15.30303        859.939394          12.424242   

                    LAYER_TYPE  MAX_MEAN_DEPTH_WIRE_REF  AADT_ALL_VEHIC  \
SHRP_ID 

i: 6
tmp:
                   VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
SHRP_ID STATE_CODE                                                         
0101    1          1995-10-30               0.6572                0.6574   
        1          1996-01-10               0.6962                0.6384   
        1          1996-04-08               0.6508                0.7148   
        1          1996-08-12               0.6804                0.7132   
        1          1996-10-11               0.6836                0.6724   
        1          1997-07-03               0.7046                0.6692   

                    MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
SHRP_ID STATE_CODE                                                        
0101    1                 88.925926        44.185185          17.454545   
        1                 88.925926        44.185185          17.454545   
        1                 88.925926        44.185185          17.454545   
      

        10                                 -1.0            337  
pad:
   VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  MAX_ANN_HUM_AVG  \
0          -1                   -1                    -1               -1   
1          -1                   -1                    -1               -1   
2          -1                   -1                    -1               -1   
3          -1                   -1                    -1               -1   
4          -1                   -1                    -1               -1   
5          -1                   -1                    -1               -1   

   MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  FREEZE_THAW_YR  TOTAL_ANN_PRECIP  \
0               -1                 -1              -1                -1   
1               -1                 -1              -1                -1   
2               -1                 -1              -1                -1   
3               -1                 -1              -1                -1   
4              

6                           -1             -1  
final:
                     VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
0                            -1              -1.0000               -1.0000   
1                            -1              -1.0000               -1.0000   
2                            -1              -1.0000               -1.0000   
3                            -1              -1.0000               -1.0000   
4                            -1              -1.0000               -1.0000   
5                            -1              -1.0000               -1.0000   
6                            -1              -1.0000               -1.0000   
(0101, 12)  1997-01-27 00:00:00               0.6858                0.9732   
(0101, 12)  2000-09-11 00:00:00               0.6606                0.8576   
(0101, 12)  2001-07-27 00:00:00               0.6932                0.8638   

            MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
0                  

4                           -1             -1  
final:
                     VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
0                            -1              -1.0000               -1.0000   
1                            -1              -1.0000               -1.0000   
2                            -1              -1.0000               -1.0000   
3                            -1              -1.0000               -1.0000   
4                            -1              -1.0000               -1.0000   
(0101, 19)  1993-10-15 00:00:00               1.0462                1.3368   
(0101, 19)  1995-02-15 00:00:00               1.2472                1.4340   
(0101, 19)  1996-04-16 00:00:00               1.4838                1.4882   
(0101, 19)  1997-09-25 00:00:00               1.5354                1.5696   
(0101, 19)  1998-10-13 00:00:00               1.5468                1.7562   

            MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
0                 -

...

(0105, 19)                   130.983333            488  
inputs:
[[ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [  1.00980002   1.05420001   0.        ]
 [ -1.          -1.         488.        ]]
i: 3
tmp:
                   VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
SHRP_ID STATE_CODE                                                         
0105    19         1993-10-15               1.0098                1.0542   
        19         1995-02-15               1.1630                1.1050   
        19         1996-04-16               1.4330                1.2376   

                    MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
SHRP_ID STATE_CODE                              

4                           -1             -1  
final:
                     VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
0                            -1              -1.0000               -1.0000   
1                            -1              -1.0000               -1.0000   
2                            -1              -1.0000               -1.0000   
3                            -1              -1.0000               -1.0000   
4                            -1              -1.0000               -1.0000   
(0105, 20)  1994-05-13 00:00:00               0.8090                0.9848   
(0105, 20)  1995-02-18 00:00:00               1.0322                1.1214   
(0105, 20)  1996-04-23 00:00:00               1.1988                1.3622   
(0105, 20)  1997-02-13 00:00:00               1.4692                1.4616   
(0105, 20)  1998-08-21 00:00:00               1.9446                2.1830   

            MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
0                  

(0105, 39)                         -1.0            135  
inputs:
[[ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [  1.06399999   1.11199997   0.        ]
 [ -1.          -1.         135.        ]]
i: 2
tmp:
                   VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
SHRP_ID STATE_CODE                                                         
0106    1          1994-08-25               0.5538                0.9402   
        1          1996-01-10               0.6028                0.5608   

                    MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
SHRP_ID STATE_CODE                                                        
0106    1                 88.925926        44.185

(0106, 10)                         -1.0            337  
inputs:
[[ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [ -1.          -1.          -1.        ]
 [  0.81639999   0.67639999   0.        ]
 [  0.7232       0.65       111.        ]
 [  0.76960001   0.6446     260.        ]
 [ -1.          -1.         337.        ]]
i: 5
tmp:
                   VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
SHRP_ID STATE_CODE                                                         
0106    10         1997-06-10               0.8164                0.6764   
        10         1997-09-29               0.7232                0.6500   
        10         1998-02-25               0.7696                0.6446   
        10         1998-05-13               0.7658                0.6528   
        10         1998-07-28               0.73

KeyboardInterrupt: 

In [11]:
raw_data = pd.read_parquet(path="../training_data/final_data.parquet")
raw_data

VISIT_DATE  IRI_LEFT_WHEEL_PATH  IRI_RIGHT_WHEEL_PATH  \
SHRP_ID STATE_CODE                                                         
0001    48         1990-03-06               0.6904                0.6570   
        48         1992-03-17               0.6894                0.7312   
        48         1993-07-13               0.7472                0.6938   
        48         1995-05-03               0.7092                0.7838   
        48         1997-08-06               0.7600                0.7508   
...                       ...                  ...                   ...   
Q350    48         1995-05-01               1.2042                1.1674   
        48         2001-04-18               1.1592                1.0578   
Q353    48         1990-09-17               1.1030                0.7784   
        48         1995-05-01               1.2294                1.0444   
        48         2001-04-18               1.1338                0.9110   

                    MAX_ANN_HUM_AVG  MIN_ANN_HUM_AVG  MEAN_ANN_TEMP_AVG  \
SHRP_ID STATE_CODE                                                        
0001    48                89.222222        44.814815          20.363636   
        48                89.222222        44.814815          20.363636   
        48                89.222222        44.814815          20.363636   
        48                89.222222        44.814815          20.363636   
        48                89.222222        44.814815          20.363636   
...                             ...              ...                ...   
Q350    48                88.333333        43.666667          18.636364   
        48                88.333333        43.666667          18.636364   
Q353    48                88.333333        43.666667          18.636364   
        48                88.333333        43.666667          18.636364   
        48                88.333333        43.666667          18.636364   

                    FREEZE_THAW_YR  TOTAL_ANN_PRECIP  TOTAL_SNOWFALL_YR  \
SHRP_ID STATE_CODE                                                        
0001    48               15.303030        859.939394          12.424242   
        48               15.303030        859.939394          12.424242   
        48               15.303030        859.939394          12.424242   
        48               15.303030        859.939394          12.424242   
        48               15.303030        859.939394          12.424242   
...                            ...               ...                ...   
Q350    48               36.295455        743.522727          15.431818   
        48               36.295455        743.522727          15.431818   
Q353    48               36.295455        743.522727          15.431818   
        48               36.295455        743.522727          15.431818   
        48               36.295455        743.522727          15.431818   

                    LAYER_TYPE  MAX_MEAN_DEPTH_WIRE_REF  AADT_ALL_VEHIC  \
SHRP_ID STATE_CODE                                                        
0001    48                   0                 5.142857       -1.000000   
        48                   0                 5.142857       -1.000000   
        48                   0                 5.142857       -1.000000   
        48                   0                 5.142857       -1.000000   
        48                   0                 5.142857       -1.000000   
...                        ...                      ...             ...   
Q350    48                   0                10.000000     1158.809524   
        48                   0                10.000000     1158.809524   
Q353    48                   0                 8.500000     1158.809524   
        48                   0                 8.500000     1158.809524   
        48                   0                 8.500000     1158.809524   

                    MEPDG_TRANS_CRACK_LENGTH_AC  
SHRP_ID STATE_CODE                               
00